In [33]:
import requests
import pandas as pd
import json
from datetime import datetime
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente
load_dotenv()

# Configurações de exibição do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Configurações da API
API_URL = os.getenv('API_URL', 'https://v3.football.api-sports.io/players')
HEADERS = {
    'x-rapidapi-key': os.getenv('API_KEY'),
    'x-rapidapi-host': os.getenv('API_HOST')
}


In [34]:

# Função para buscar dados do jogador
def get_player_data(player_id):
    params = {
        'id': player_id,
        'season': '2023'
    }
    response = requests.get(API_URL, headers=HEADERS, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None


In [35]:

# Buscar dados do Coutinho
coutinho_data = get_player_data(147)

print("Dados brutos da API:")
print(json.dumps(coutinho_data, indent=2))

if coutinho_data and 'response' in coutinho_data:
    player_info = coutinho_data['response'][0]
    
    # Dados do jogador
    player_df = pd.json_normalize(player_info['player'])
    
    # Dados estatísticos
    stats_df = pd.json_normalize(player_info['statistics'])


Dados brutos da API:
{
  "get": "players",
  "parameters": {
    "id": "147",
    "season": "2023"
  },
  "errors": [],
  "results": 1,
  "paging": {
    "current": 1,
    "total": 1
  },
  "response": [
    {
      "player": {
        "id": 147,
        "name": "Philippe Coutinho",
        "firstname": "Philippe",
        "lastname": "Coutinho Correia",
        "age": 33,
        "birth": {
          "date": "1992-06-12",
          "place": "Rio de Janeiro",
          "country": "Brazil"
        },
        "nationality": "Brazil",
        "height": "172 cm",
        "weight": "68 kg",
        "injured": false,
        "photo": "https://media.api-sports.io/football/players/147.png"
      },
      "statistics": [
        {
          "team": {
            "id": 2904,
            "name": "Al-Duhail SC",
            "logo": "https://media.api-sports.io/football/teams/2904.png"
          },
          "league": {
            "id": 305,
            "name": "Stars League",
            "country

In [36]:

    # Filtrar por ligas específicas
stars_league = stats_df[stats_df['league.name'] == 'Stars League']
premier_league = stats_df[stats_df['league.name'] == 'Premier League']

In [37]:
stars_league

,team.id,team.name,team.logo,league.id,league.name,league.country,league.logo,league.flag,league.season,games.appearences,games.lineups,games.minutes,games.number,games.position,games.rating,games.captain,substitutes.in,substitutes.out,substitutes.bench,shots.total,shots.on,goals.total,goals.conceded,goals.assists,goals.saves,passes.total,passes.key,passes.accuracy,tackles.total,tackles.blocks,tackles.interceptions,duels.total,duels.won,dribbles.attempts,dribbles.success,dribbles.past,fouls.drawn,fouls.committed,cards.yellow,cards.yellowred,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved
0,2904,Al-Duhail SC,https://media.api-sports.io/football/teams/2904.png,305,Stars League,Qatar,https://media.api-sports.io/football/leagues/305.png,https://media.api-sports.io/flags/qa.svg,2023,16,12,1081,None,Midfielder,7.675000,False,4,10,4,5.0,2.0,3,0.0,1.0,None,153.0,10.0,None,None,1.0,1.0,20.0,10.0,9.0,6.0,None,3.0,None,0,0,0,None,None,0.0,0.0,None


In [38]:
premier_league

,team.id,team.name,team.logo,league.id,league.name,league.country,league.logo,league.flag,league.season,games.appearences,games.lineups,games.minutes,games.number,games.position,games.rating,games.captain,substitutes.in,substitutes.out,substitutes.bench,shots.total,shots.on,goals.total,goals.conceded,goals.assists,goals.saves,passes.total,passes.key,passes.accuracy,tackles.total,tackles.blocks,tackles.interceptions,duels.total,duels.won,dribbles.attempts,dribbles.success,dribbles.past,fouls.drawn,fouls.committed,cards.yellow,cards.yellowred,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved
1,66,Aston Villa,https://media.api-sports.io/football/teams/66.png,39,Premier League,England,https://media.api-sports.io/football/leagues/39.png,https://media.api-sports.io/flags/gb-eng.svg,2023,2,0,24,None,Midfielder,6.800000,False,2,1,2,3.0,1.0,0,0.0,NaN,None,24.0,NaN,None,None,NaN,NaN,7.0,5.0,5.0,4.0,None,1.0,None,0,0,0,None,None,0.0,0.0,None


In [30]:
# Importar as funções do app.py
from app import prepare_data_for_api

# Preparar dados para análise
stars_league_clean = prepare_data_for_api(stars_league)
premier_league_clean = prepare_data_for_api(premier_league)

# Exibir as métricas calculadas
print("Métricas calculadas para Stars League:")
for col in stars_league_clean.columns:
    if col not in stars_league.columns:
        print(f"{col}: {stars_league_clean[col].values[0]}")

print("\nMétricas calculadas para Premier League:")
for col in premier_league_clean.columns:
    if col not in premier_league.columns:
        print(f"{col}: {premier_league_clean[col].values[0]}")


In [31]:
# Importar a função format_for_api do app.py
from app import format_for_api

# Formatar dados para API
stars_league_api = format_for_api(player_info, stars_league_clean)
premier_league_api = format_for_api(player_info, premier_league_clean)

# Verificar estrutura dos dados formatados
print("Estrutura dos dados formatados para API:")
print("- Player info")
for key in stars_league_api['player'].keys():
    print(f"  - {key}")
print("\n- Stats categories:")
for key in stars_league_api['stats'].keys():
    print(f"  - {key}")


In [39]:
# Testar o endpoint implementado no app.py
import requests

def test_api_endpoint():
    """
    Testa o endpoint implementado no app.py
    """
    # Iniciar o servidor Flask em um processo separado antes de testar
    try:
        response = requests.get('http://localhost:5000/api/player/147/stats')
        if response.status_code == 200:
            print("API endpoint funcionando corretamente!")
            data = response.json()
            print(f"Dados do jogador: {data['player']['name']}")
            print(f"Ligas disponíveis: {list(data['leagues'].keys())}")
            return data
        else:
            print(f"Erro ao acessar o endpoint: {response.status_code}")
            return None
    except requests.exceptions.ConnectionError:
        print("Não foi possível conectar ao servidor Flask. Verifique se o servidor está rodando.")
        return None

# O teste pode ser executado após iniciar o servidor Flask com 'python app.py'
# test_api_endpoint()
